# This program can plot u_e_ij for a $10 \times 10$ mesh.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

from matplotlib import pyplot as plt
import numpy as np
import arrayfire as af
import h5py

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh
from dg_maxwell import wave_equation_2d
from dg_maxwell import lagrange

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

[[ 0  9]
 [10 19]
 [20 29]
 [30 39]
 [40 49]
 [50 59]
 [60 69]
 [70 79]
 [80 89]
 [90 99]]
[[ 0 90]
 [ 1 91]
 [ 2 92]
 [ 3 93]
 [ 4 94]
 [ 5 95]
 [ 6 96]
 [ 7 97]
 [ 8 98]
 [ 9 99]]
gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


In [2]:
params.mesh_file = '../read_and_plot_mesh/mesh/square_10_10.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)
new_elements = msh_parser.rearrange_element_edges(advec_var.elements, advec_var)
advec_var.reassign_2d_elements(new_elements)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 441 nodes.
gmshTranslator: Mesh has 144 elements.
gmshTranslator: Processed 589 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


In [3]:
def read_1d_ME_hdf5(index):
    '''
    '''
    h5py_data = h5py.File('../../results/hdf5_%02d/dump_timestep_%06d' \
                          %(int(params.N_LGL), int(20 * index)) + '.hdf5', 'r')
    E_z_LGL     = (h5py_data['E_z'][:])
    B_y_LGL     = (h5py_data['B_y'][:])

    return (E_z_LGL, B_y_LGL)

def plot_1d_ME(u_1d, advec_var, ax):
    '''
    '''
    ax.plot(advec_var.element_LGL, u_1d, '.')
#     plt.show()


In [4]:
def contour_2d(u, advec_var, ax):
    '''
    '''
    color_levels = np.linspace(-1.0, 1.1, 100)
    u_plot = af.flip(af.moddims(u, params.N_LGL, params.N_LGL, 10, 10), 0)
    x_plot = af.flip(af.moddims(advec_var.x_e_ij, params.N_LGL, params.N_LGL, 10, 10), 0)
    y_plot = af.flip(af.moddims(advec_var.y_e_ij, params.N_LGL, params.N_LGL, 10, 10), 0)


    x_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    y_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    u_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    #
    for r in range(100):
        p = int(r / 10)
        q = r - p * 10
        x_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = x_plot[:, :, q, p]

        y_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = y_plot[:, :, q, p]

        u_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = u_plot[:, :, q, p]

    x_contour = np.array(x_contour)
    y_contour = np.array(y_contour)
    u_contour = np.array(u_contour)

    im = ax.plot(y_contour[:, 55], u_contour[:, 55])
    
#     ax.set_aspect('equal')
#     axes.title('Time = %.2f' %(index * 10 * advec_var.delta_t_2d))
#     fig.savefig('../../results/2D_Wave_images/%04d' %(index) + '.png')
#     plt.close('all')
#     plt.show()
    return im

def u_mesh_2d(u, advec_var):
    '''
    '''
    u_plot = af.flip(af.moddims(u, params.N_LGL, params.N_LGL, 10, 10), 0)
    x_plot = af.flip(af.moddims(advec_var.x_e_ij, params.N_LGL, params.N_LGL, 10, 10), 0)
    y_plot = af.flip(af.moddims(advec_var.y_e_ij, params.N_LGL, params.N_LGL, 10, 10), 0)


    x_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    y_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    u_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    #
    for r in range(100):
        p = int(r / 10)
        q = r - p * 10
        x_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = x_plot[:, :, q, p]

        y_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = y_plot[:, :, q, p]

        u_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = u_plot[:, :, q, p]

    x_contour = np.array(x_contour)
    y_contour = np.array(y_contour)
    u_contour = np.array(u_contour)

    return x_contour, y_contour, u_contour

gmshTranslator: Ending


In [5]:
results_directory = '../../results/2D_Wave_images'

if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# The directory where h5py files are stored.
h5py_directory = '../../results/2d_hdf5_%02d' %int(params.N_LGL)
path, dirs, files = os.walk(h5py_directory).__next__()
file_count = len(files)
print(h5py_directory)
print(file_count)

shape_u = utils.shape(advec_var.u_e_ij)

for i in np.arange(file_count):
    print(i)
    fig = plt.figure()

    h5py_data = h5py.File('../../results/2d_hdf5_%02d/dump_timestep_%06d' % (int(params.N_LGL), int(i * 20)) + '.hdf5', 'r')
    u_LGL     = af.np_to_af_array(h5py_data['u_i'][:])

    E_z, B_y = read_1d_ME_hdf5(i)

    ax1 = plt.subplot2grid((2, 1), (0, 0))
    ax2 = plt.subplot2grid((2, 1), (1, 0))

    contour_2d(u_LGL[:, :, 0], advec_var, ax1)
    plot_1d_ME(E_z, advec_var, ax1)

    ax1.set_xlabel(r'$y$', fontsize = 12)
    ax1.set_ylabel(r'$E_z$', fontsize = 12)
    ax1.set_title(label = r'$E_z(t = %.2f)$'%(i * 20 * advec_var.delta_t_2d), fontsize = 12)

    contour_2d(u_LGL[:, :, 1], advec_var, ax2)
    plot_1d_ME(-B_y, advec_var, ax2)

    ax2.set_xlabel(r'$y$', fontsize = 12)
    ax2.set_ylabel(r'$B_y$', fontsize = 12)
    ax2.set_title(label = r'$B_x(t = %.2f)$'%(i * 20 * advec_var.delta_t_2d), fontsize = 12)

    # # im = contour_2d(u_LGL[:, :, 2], advec_var, ax3, title = r'$B_y(t = %.2f)$'%(i * 20 * advec_var.delta_t_2d))


    fig.suptitle('Plot Showing comparision b/w $1D$ and $2D$ \n' \
                 + r'simulation of Maxwell\'s Equations.\n' \
                 + r'`o` for $1D$ results and `-` for 2D results',
                 fontsize=14, y = 1.03)

    plt.tight_layout()

    fig.savefig('../../results/1d_2d_ME_compare_img/%04d' %(i) + '.png', bbox_inches='tight', dpi = 300)
    plt.close('all')
# plt.show()

../../results/2d_hdf5_08
157
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

# Error plot between $1D$ and $2D$ wave equations

In [176]:
results_directory = '../../results/2D_Wave_images'

if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# The directory where h5py files are stored.
h5py_directory = '../../results/2d_hdf5_%02d' %int(params.N_LGL)
path, dirs, files = os.walk(h5py_directory).__next__()
file_count = len(files)
print(h5py_directory)
print(file_count)

shape_u = utils.shape(advec_var.u_e_ij)

# for i in np.arange(file_count):
i = 78
print(i)
fig = plt.figure()

h5py_data = h5py.File('../../results/2d_hdf5_%02d/dump_timestep_%06d' % (int(params.N_LGL), int(i * 20)) + '.hdf5', 'r')
u_LGL     = af.np_to_af_array(h5py_data['u_i'][:])

E_z, B_y = read_1d_ME_hdf5(i)

E_z = af.flat(af.np_to_af_array(E_z))
element_LGL = af.flat(advec_var.element_LGL)
# print(element_LGL)

x_mesh, y_mesh, u_mesh = u_mesh_2d(u_LGL[:, :, 0], advec_var)
print(np.abs(np.flipud(u_mesh)[:, 55] - np.array(E_z)))
# print(np.flipud(y_mesh))
# for i, iterator in enumerate(element_LGL):
#     print((np.flipud(y_mesh)[i, 55]-af.sum(iterator)))

print(np.e**(-(y_mesh[:, 55])**2 / 0.4**2))
print(i * 20 * advec_var.delta_t_2d)
# plt.plot(y_mesh[:, 55], u_mesh[:, 55], '-')
# plt.plot(np.array(element_LGL), np.array(E_z), '*')
# plt.show()

../../results/2d_hdf5_08
79
78
[  4.84898363e-10   2.41856721e-10   1.17253411e-10   3.83820945e-10
   3.70384320e-10   5.07387405e-10   2.69841323e-10   6.07202195e-10
   1.30472597e-09   9.55768083e-10   1.44586079e-11   1.93667155e-09
   2.72028459e-09   4.50034024e-09   1.55877306e-09   2.47388267e-09
   5.32180108e-09   2.42496813e-09   1.49241075e-09   9.59929844e-09
   9.66087693e-09   1.19460059e-08   6.70662420e-09   1.23424590e-08
   6.18317758e-09   8.77870338e-09   1.02009284e-08   2.95598002e-09
   9.80494563e-09   1.81529913e-09   1.47683831e-08   4.26553226e-09
   2.12002099e-08   3.36635475e-09   6.97683911e-10   5.93740535e-09
   1.72539920e-08   5.09724463e-09   2.93754154e-09   1.77036013e-08
   7.68243491e-09   6.12860540e-09   7.04564940e-09   1.41356478e-08
   2.13700276e-08   5.69650949e-09   1.70871817e-08   3.97851052e-09
   2.18341902e-08   1.56776867e-08   1.84023137e-08   2.85329760e-10
   1.06033009e-08   1.91052986e-08   1.37298558e-08   8.25213176e-10
   